# Step 1: upload the folder AUAOI to your google drive

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [0]:
PATH = "drive/My Drive/AUAOI"
!ls "drive/My Drive/AUAOI"

# Step 2: Using Sikit-learn and Keras

In [0]:
import numpy as np
import os
import time

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model


# Step 3: Read images

In [0]:
#Setting data path
#Read the file names in the folder directory
### START CODE HERE ###  (≈ 2 lines)
data_path = PATH + '/AOIdata_100'
data_dir_list = os.listdir(data_path)
### END CODE HERE ###

#設定一個list
img_data_list=[]

#Read images under the folders with loops 
### START CODE HERE ###  (≈ 5 lines)
for dataset in data_dir_list:
  img_list=os.listdir(data_path+'/'+dataset)
  print ('Loaded the images of dataset-'+dataset)
  for img in img_list:
    img_path = data_path + '/' + dataset + '/' + img
### END CODE HERE ###
    
    #Read the image 
    #Convert the image to array 
    #Add the dimension to the front of the array 
    #Preprocess_input
    #Save the array to the list
    ### START CODE HERE ###  (≈ 5 lines)
    img = image.load_img(img_path)
    x=image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    img_data_list.append(x)
    ### END CODE HERE ###

img_data = np.array(img_data_list)

print (img_data.shape)  #(600, 1, 224, 224, 3)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)  #(1, 600, 224, 224, 3)
img_data=img_data[0]
print (img_data.shape)  #(600, 224, 224, 3)

# Step 4: Labeling the images

In [0]:
# Labeling the images
### START CODE HERE ###  (≈ 3 lines)
num_classes = 6
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
### END CODE HERE ###

labels[0:100]=0
labels[100:200]=1
labels[200:300]=2
labels[300:400]=3
labels[400:500]=4
labels[500:600]=5

# one-hot encoding
###  CODE HERE ###  (≈ 1 lines)
Y = np_utils.to_categorical(labels, num_classes)

#Shuffling dataset
###  CODE HERE ###  (≈ 1 lines)
x,y = shuffle(img_data,Y, random_state=2)

# Split images into training set and test set
###  CODE HERE ###  (≈ 1 lines)
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=2)


# Step5: Customize VGG16 model

In [0]:
#input shape
###  CODE HERE ###  (≈ 1 lines)
image_input = Input(shape=(224, 224, 3))

#Use the VGG16 model 
###  CODE HERE ###  (≈ 1 lines)
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

#Summary of the customize VGG16 model
###  CODE HERE ###  (≈ 1 lines)
model.summary()


last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
	layer.trainable = False

#Set the model's loss function, optimizer, and metrics
###  CODE HERE ###  (≈ 1 lines)
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t=time.time()
#The parameters for the training model
###  CODE HERE ###  (≈ 1 lines)
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

print('Training time: %s' % time.time())

#Assess the accuracy of the trained model to the test data
###  CODE HERE ###  (≈ 1 lines)
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)


print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

#Preditction of the test dataset
#Print out the predicted Label value
### START CODE HERE ###  (≈ 5 lines)
prediction = custom_vgg_model.predict(X_test)
predict = np.argmax(prediction,axis=1)
ans =np.argmax(y_test,axis=1)
print(predict)
print(ans)
### END CODE HERE ###



# Step 6: Output loss and accuracy figures

In [0]:
Read 60 images in AOIdata_test and use trained models for prediction#%%
# %matplotlib notebook
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(10)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.show()


# Exercise: 
Read 60 images in AOIdata_test and use trained models for prediction